In [ ]:
day_obs = "yesterday"
show_joined = False

# Identify potential bad images for {{ params.day_obs }}

This notebook uses information from the ConsDb and the EFD to try to identify potential bad visits.

Visits acquired from the FeatureBasedScheduler are tracked in the observatory system through several stages:
* a target event is issued when the visit is requested
* when this requested target is next to be observed, a nextvisit event is issued
* an observation event is issued when the script to acquire the event is registered as successful by the Scheduler
* as the visit is processed and metadata lands in the ConsDb, visit information including zeropoints measured by Rapid Analysis (RA) become available

These can be linked: target + observation are linked by `target_id`, target + nextvisit are linked by the `script sal index`, and nextvisit + consdb visit can be linked by `group_id`.

If the target is requested and visits are acquired, but the observation is never registered, then the script may have been interrupted  by a fault which would also cause problems for the visit.

If the zeropoint reported by RA is significantly far from the predicted zeropoint (currently using an ad-hoc per-visit magnitude cutoff), then it is likely that there were problems with the visit.

In each of these cases, we can identify the potentially troublesome visits and then check by eye in RubinTV -- links are provided to the full mosaic and the witness detector (at USDF).

---------

In [ ]:
!pip install --user --upgrade git+https://github.com/lsst-sims/rubin_nights.git  --no-deps  > /dev/null 2>&1

In [ ]:
import rubin_nights
print('rubin_nights version', rubin_nights.__version__)

In [ ]:
import os
import numpy as np
import pandas as pd
from astropy.time import Time, TimeDelta
#from astroplan import Observer

from rubin_nights import connections, scriptqueue
    
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, HTML

import logging
logging.getLogger('rubin_nights').setLevel(logging.INFO)

----------

In [ ]:
if os.getenv("EXTERNAL_INSTANCE_URL") is None:
    tokenfile = '/Users/lynnej/.lsst/usdf_rsp'
    site = 'usdf'
    endpoints = connections.get_clients(tokenfile=tokenfile, site=site)
else:
    tokenfile = None
    site = None
    endpoints = connections.get_clients()

In [ ]:
if day_obs.lower() == "today":
    day_obs = Time(Time.now().mjd - 0.5, format='mjd', scale='tai').iso[0:10]
elif day_obs.lower() == "yesterday":
    day_obs = Time(Time.now().mjd - 1.5, format='mjd', scale='tai').iso[0:10]
else:
     day_obs = day_obs
    
day_obs_time = Time(f"{day_obs}T12:00:00", format='isot', scale='tai')
#observer = Observer.at_site('lsst')
#sunset = Time(observer.sun_set_time(day_obs_time, which='next', horizon=-10*u.deg), format='jd')
#sunrise = Time(observer.sun_rise_time(day_obs_time, which='next', horizon=-10*u.deg), format='jd')
sunset = Time(f"{day_obs}T12:00:00", format='isot', scale='tai')
sunrise = Time(f"{day_obs}T12:00:00", format='isot', scale='tai') + TimeDelta(1, format='jd')
print(f"Checking night time on {day_obs}, from {sunset.iso} to {sunrise.iso}")
print(f"Time of notebook execution {Time.now().iso}")

topic = "lsst.sal.Scheduler.logevent_target"
fields = ['targetId', 'airmass', 'alt', 'az', 'blockId', 'cloud', 'decl',
       'exposureTimes0', 'exposureTimes1', 'exposureTimes2', 'exposureTimes3',
       'exposureTimes4', 'exposureTimes5', 'exposureTimes6', 'exposureTimes7',
       'exposureTimes8', 'exposureTimes9', 'filter', 'isSequence', 'moonAlt',          
       'moonAz', 'moonDec', 'moonDistance', 'moonPhase', 'moonRa', 'note',
       'numExposures', 'numProposals', 'offsetX', 'offsetY',
       'ra', 'requestMjd',
       'requestTime', 'rotAngle', 'salIndex', 'schedulerNote', 'seeing',
       'sequenceDuration', 'sequenceNVisits', 'sequenceVisits', 'skyAngle',
       'skyBrightness', 'slewTime', 'snapshotUri', 'solarElong', 'sunAlt',
       'sunAz', 'sunDec', 'sunRa', 'targetName']
#topic = "lsst.sal.Scheduler.logevent_largeFileObjectAvailable"
targets = endpoints['efd'].select_time_series(topic, fields, sunset, sunrise, index=1)
print(f"{len(targets)} targets events")

topic = "lsst.sal.Scheduler.logevent_observation"
fields = ['additionalInformation', 'blockId', 'decl', 'exptime', 'filter', 'mjd', 'nexp', 'ra', 'rotSkyPos', 'salIndex', 'targetId']
observations = endpoints['efd'].select_time_series(topic, fields, sunset, sunrise, index=1)
if len(observations) == 0:
    observations = pd.DataFrame([], columns=fields + ['time'])
print(f"{len(observations)} observation events")

topic = "lsst.sal.ScriptQueue.logevent_nextVisit"
nextvisits = endpoints['efd'].select_time_series(topic, '*', sunset, sunrise, index=1)
nextvisits = nextvisits.query("survey in 'BLOCK-365'")
print(f"{len(nextvisits)} next visit events")

#visits = endpoints['consdb_tap'].get_visits('lsstcam', sunset, sunrise)
visits = endpoints['consdb'].get_visits('lsstcam', sunset, sunrise)
visits = visits.query("science_program == 'BLOCK-365'")
print(f"{len(visits)} visits for science_program BLOCK-365")

In [ ]:
# In theory, targets and observations could be merged directly on targetId
# However, targetId is not yet unique across FBS re-enable times 
# (the observations will have unique targetId, but targets which were not actually attempted may be duplicate targetIds)
if len(targets) == 0:
    joint = pd.DataFrame([], columns=['targetId', 'blockId', 'skyAngle'])
    joint = joint.astype({'targetId': int, 'blockId': int, 'skyAngle': float})
elif len(observations) == 0:
    new_df = pd.DataFrame(np.zeros((len(targets.index.values), len(observations.columns.values))),
                           columns=observations.columns.values, index=targets.index)
    new_df.rename({'time': 'time_o'}, axis=1, inplace=True)
    new_df.time_o = np.nan
    joint = pd.merge(targets, 
                     new_df, 
                     left_index=True, right_index=True, suffixes=('', '_o'))
    joint.reset_index('time', inplace=True)
    print(f"Added empty observations to {len(joint)} joined targets+empty observations")
else:
    joint = pd.merge_asof(targets.sort_values('targetId').reset_index('time'), 
                       observations.sort_values('targetId').reset_index('time'),
                       on='targetId', 
                       left_by=['ra', 'decl', 'skyAngle'], right_by=['ra', 'decl', 'rotSkyPos'],
                       suffixes=('', '_o'), allow_exact_matches=True, direction='forward')
    joint.sort_values(by='time', inplace=True)
    print(f"Joined targets and observations for {len(joint)} events")

In [ ]:
# jcols = ['time', 'time_o', 'targetId', 'blockId', 'ra', 'decl', 'skyAngle', 'rotSkyPos', 'filter', 'exptime', 'note', 'mjd','requestTime', 'snapshotUri']
# joint[jcols]

In [ ]:
# And nextVisit to visits groupId should be unique 
nv = pd.merge(visits, nextvisits.reset_index('time'), 
              how='outer', left_on='group_id', right_on='groupId', suffixes=['', '_nv'])
visit_id = np.where(np.isnan(nv['visit_id'].values), 0, nv['visit_id'].values)
nv['visit_id'] = visit_id
scriptSalIndex = np.where(np.isnan(nv['scriptSalIndex'].values), 0, nv['scriptSalIndex'].values)
nv['scriptSalIndex'] = scriptSalIndex
nv = nv.astype({'visit_id': int, 'scriptSalIndex': int, 'cameraAngle': float})
print(f"Joined nextvisit and visits for {len(nv)} records")

nvcols = ['visit_id', 'time', 'obs_start', 'position0', 'position1', 'cameraAngle', 's_ra', 's_dec', 'sky_rotation', 'band', 'science_program', 'target_name', 'scriptSalIndex']
# nv.query('visit_id > 0')[nvcols]
# nv[nvcols]

In [ ]:
# But targets blockId (== salScriptId) to nextVisits salScriptId is only unique within times of scriptqueue restarts
# We can narrow down the links using the positions of the fields maybe? 

vt = pd.merge_asof(joint.sort_values('blockId'), nv.sort_values('scriptSalIndex'), 
                   left_on='blockId', right_on='scriptSalIndex', suffixes=['', '_nv'], 
                  left_by=['skyAngle'], right_by=['cameraAngle'], 
                   allow_exact_matches=True, direction='forward')
int_cols = ['visit_id', 'day_obs', 'seq_num', 'scriptSalIndex']
for col in int_cols:
    tt = np.where(np.isnan(vt[col].values), 0, vt[col].values)
    vt[col] = tt
vt = vt.astype(dict([(col, int) for col in int_cols]))
vt.sort_values('time', inplace=True)
print(f"Joined targets+observations with nextvisit+visit for {len(vt)} records")

vt.rename({'time': 'time_target', 
           'time_o': 'time_observation',
           'time_nv': 'time_nextvisit'},
          axis=1, inplace=True)
cols = ['visit_id', 'day_obs', 'seq_num', 'time_target', 'time_observation', 'time_nextvisit', 'obs_start', 'group_id', 
        'ra', 'decl', 'skyAngle', 's_ra', 's_dec', 'sky_rotation', 'band', 'scriptSalIndex', 
        'note', 'observation_reason', 'target_name']
if show_joined:
    display(HTML(vt[cols].to_html()))

In [ ]:
# Were there visits where the rotation angle was changed after the request/next visit?
changed_rot = vt[abs(vt.skyAngle - vt.sky_rotation) > 1]
if len(changed_rot) > 0:
    display(changed_rot[cols])
else:
    print("No visits with mismatch in sky_rotation compared to skyAngle in target request.")

In [ ]:
rubintv_base = "https://usdf-rsp.slac.stanford.edu/rubintv/summit-usdf/lsstcam/event?key=lsstcam/"
def rubintv_links(day_obs, seq_num):
    witness_link = f"{rubintv_base}{day_obs}/witness_detector/{seq_num :06d}/lsstcam_witness_detector_{day_obs}_{seq_num :06d}.jpg"
    witness_html = f'<a href="{witness_link}" target="_blank" rel="noreferrer noopener">{witness_link}</a>'
    mosaic_link = f"{rubintv_base}{day_obs}/calexp_mosaic/{seq_num :06d}/lsstcam_calexp_mosaic_{day_obs}_{seq_num :06d}.jpg"
    mosaic_html = f'<a href="{mosaic_link}" target="_blank" rel="noreferrer noopener">{mosaic_link}</a>'
    return mosaic_html, witness_html

In [ ]:
pcols = ['visit_id', 'day_obs', 'seq_num', 'time_target', 'time_observation', 'obs_start', 'scriptSalIndex', 
          'target_name', 's_ra', 's_dec', 'sky_rotation', 'band', 
         'zero_point_1s', 'zero_point_1s_pred',]

if len(vt) == 0:
    print("No observations.")

else:
    quicklook_missing = np.where(np.isnan(vt.zero_point_median) & (vt.visit_id > 0))[0]
    big_zp_offset = np.where((vt.zero_point_1s_pred.values - vt.zero_point_1s.values) > 1)[0]
    failed_obs = np.where(np.isnan(vt.time_observation.values) & vt.visit_id > 0)[0]
    issues = np.concatenate([quicklook_missing, big_zp_offset, failed_obs])
    issues = np.sort(issues)
    issues = np.unique(issues)

    if len(issues) == 0:
        print("No obvious issues found.")
    else:
        display(HTML("<hr>"))
        display(HTML("Potential problems."))
        display(HTML("<br>"))
        for idx in issues:
            problem_string = []
            if idx in failed_obs:
                problem_string.append("no observation event")
            if idx in quicklook_missing:
                problem_string.append("no quicklook")
            if idx in big_zp_offset:
                problem_string.append("big zeropoint offset")
            problem_string = ", ".join(problem_string)
            display(pd.DataFrame(vt.iloc[idx][pcols]).T)
            print(problem_string)
            mosaic_html, witness_html = rubintv_links(day_obs, vt.iloc[idx].seq_num)
            display(HTML(mosaic_html))
            display(HTML(witness_html))
            display(HTML("<br>"))
        display(HTML("<hr>"))

In [ ]:
if len(failed_obs) > 0:
    efd_and_messages, ecols = scriptqueue.get_consolidated_messages(sunset, sunrise, tokenfile=tokenfile, site=site)
    scols = ['visit_id', 'day_obs', 'seq_num', 'time', 'time_o', 'time_nv', 'groupId', 'obs_start', 
        'ra', 'decl', 'skyAngle', 'sky_rotation', 'band', 'scriptSalIndex', 
        'note', 'target_name']
    mtimes = pd.to_datetime(efd_and_messages.time)
    for idx in failed_obs:
        display(HTML("<hr>"))
        display(HTML("ScriptQueue info on visits missing observations (cause of failure?)"))
        display(HTML("<br>"))
        display(pd.DataFrame(vt.iloc[idx][pcols]).T)
        display(HTML("ScriptQueue Information with same scriptSalIndex"))
        fo = vt.iloc[idx]
        fo_time = pd.to_datetime(fo.time_target)
        mm = efd_and_messages.iloc[np.where(abs(mtimes - fo_time) < pd.Timedelta(1, unit='hr'))[0]]
        mm = mm.query('script_salIndex == @fo.scriptSalIndex')
        display(HTML(mm[ecols].to_html()))
        display(Markdown("--"))

In [ ]:
# Targets which did not result in a visit
if len(vt) > 0:
    incomplete_target = vt.query('visit_id == 0')[cols]
    if len(incomplete_target) == 0:
        print("All requested targets resulted in visits.")
    else:
        print("Targets requested by not observed.")
        display(HTML(incomplete_target.to_html()))